In [2]:
import pandas as pd

In [91]:
#ser_ts_sum = train[train['trx_category'] == 'POS'][['cl_id', 'target_sum', 'year', 'month']].groupby(by=['cl_id', 'year', 'month']).sum().astype('int64')
#ser_pos_count = train[train['trx_category'] == 'POS'][['cl_id', 'target_sum', 'year', 'month']].groupby(by=['cl_id', 'year', 'month']).count().rename(columns={'target_sum':'count_POS'}).astype('int64')
#ser_notpos_count = train[train['trx_category'] != 'POS'][['cl_id', 'target_sum', 'year', 'month']].groupby(by=['cl_id', 'year', 'month']).count().rename(columns={'target_sum':'count_notPOS'}).astype('int64')
#train = train.drop(axis=1, labels='target_sum').join(ser_ts_sum, on=['cl_id', 'year', 'month'], how='left').join(ser_pos_count, on=['cl_id', 'year', 'month'], how='left').join(ser_notpos_count, on=['cl_id', 'year', 'month'], how='left')


'''
[description]
InsertDataFrame - pandas dataframe
ColumnName - name of transforming column
'''
def MakeDateYearMonth(InsertDataFrame, ColumnName):
    df = InsertDataFrame.copy()
    df[ColumnName] = pd.to_datetime(df[ColumnName])
    df['year'] = df[ColumnName].dt.year
    df['month'] = df[ColumnName].dt.month
    return df    

'''
[description]
InsertDataFrame - pandas dataframe
GroupByList - list of column names
TargetColumn - name of one column in dataframe, string
TargetName - name of returning column
'''
def groupbysum(InsertDataFrame, GroupByList, TargetColumn, TargetName):
    df = InsertDataFrame.copy()
    sdtype = df[TargetColumn].dtype
    ser = df.groupby(by=GroupByList).sum().rename(columns={TargetColumn:TargetName}).astype(sdtype)
    return ser
    
'''
[description]
InsertDataFrame - pandas dataframe
GroupByList - list of column names
TargetColumn - name of one column in dataframe, string
TargetName - name of returning column
'''
def groupbycount(InsertDataFrame, GroupByList, TargetColumn, TargetName):
    df = InsertDataFrame.copy()
    ser = df.groupby(by=GroupByList).count().rename(columns={TargetColumn:TargetName}).astype('int64')
    return ser    

'''
[description]
InsertDataFrame - pandas dataframe
SeriesList - list of pandas series
OnJoin - string or list of column name(s)
'''
def DfLeftJoinSer(InsertDataFrame, SeriesList, OnJoin):
    df = InsertDataFrame.copy()
    for i in SeriesList:
        df = df.join(i, on=OnJoin, how='left')
    return df

In [56]:
url_train = 'https://raw.githubusercontent.com/Hitriihoma/Machine-learning-course-task/master/train.csv'
url_test = 'https://raw.githubusercontent.com/Hitriihoma/Machine-learning-course-task/master/test.csv'

train = pd.read_csv(url_train)
test = pd.read_csv(url_test)

In [57]:
train2 = train.copy()

In [92]:
train = train2.copy()
train.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0.0


In [93]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490513 entries, 0 to 490512
Data columns (total 9 columns):
PERIOD          490513 non-null object
cl_id           490513 non-null int64
MCC             490513 non-null int64
channel_type    487603 non-null object
currency        490513 non-null int64
TRDATETIME      490513 non-null object
amount          490513 non-null float64
trx_category    490513 non-null object
target_sum      490513 non-null float64
dtypes: float64(2), int64(3), object(4)
memory usage: 33.7+ MB


# Метрики для модели
год, месяц - сколько берём? надо смотреть сколько есть  
количество POS-транзакций  
количество не-POS-транзакций  
сумма POS-транзакций  
сумма не- POS-транзакций

In [95]:
# Входные сырые данные + колонки
train = MakeDateYearMonth(train, 'PERIOD')
ser_ts_sum = groupbysum(train[train['trx_category'] == 'POS'][['cl_id', 'target_sum', 'year', 'month']],
                       ['cl_id', 'year', 'month'],
                       'target_sum',
                       'target_sum')
ser_pos_amount_sum = groupbysum(train[train['trx_category'] == 'POS'][['cl_id', 'amount', 'year', 'month']],
                       ['cl_id', 'year', 'month'],
                       'amount',
                       'pos_sum')
ser_notpos_amount_sum = groupbysum(train[train['trx_category'] != 'POS'][['cl_id', 'amount', 'year', 'month']],
                       ['cl_id', 'year', 'month'],
                       'amount',
                       'notpos_sum')
ser_pos_count = groupbycount(train[train['trx_category'] == 'POS'][['cl_id', 'target_sum', 'year', 'month']],
                       ['cl_id', 'year', 'month'],
                       'target_sum',
                       'count_pos')
ser_notpos_count = groupbycount(train[train['trx_category'] != 'POS'][['cl_id', 'target_sum', 'year', 'month']],
                       ['cl_id', 'year', 'month'],
                       'target_sum',
                       'count_notpos')

train = DfLeftJoinSer(train.drop(axis=1, labels='target_sum'),
                      [ser_ts_sum, ser_pos_amount_sum, ser_notpos_amount_sum, ser_pos_count, ser_notpos_count],
                      ['cl_id', 'year', 'month'])
train.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,year,month,target_sum,pos_sum,notpos_sum,count_pos,count_notpos
0,2017-01-10,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,2017,1,0.0,7821.0,56562.0,3.0,2.0
1,2017-01-10,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,2017,1,0.0,7821.0,56562.0,3.0,2.0
2,2017-01-12,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,2017,1,0.0,7821.0,56562.0,3.0,2.0
3,2017-01-10,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,2017,1,0.0,7821.0,56562.0,3.0,2.0
4,2017-01-10,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,2017,1,0.0,7821.0,56562.0,3.0,2.0


# Анализ данных
какой месяц для этого текущий  
сколько для каждого клиента известно месяцев

In [103]:
# количество уникальных клиентов
len(train.cl_id.unique())

5000

# Формирование выборки

In [63]:
# Пример формирования тестовой выборки. Возьмем 10 случайных пользвоателей.
users_sample = train['cl_id'].sample(10).values
user_sample_df = train[(train['cl_id'].isin(users_sample))][:]
user_sample_df.head(5)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,year,month,target_sum,count_POS,count_notPOS
48794,2017-01-04,901,5814,type2,810,29APR17:00:00:00,402.0,POS,2017,1,58147131.81,223.0,17.0
48795,2017-01-04,901,5261,type2,810,15APR17:00:00:00,28490.5,POS,2017,1,58147131.81,223.0,17.0
48799,2017-01-03,901,5921,type2,810,03MAR17:00:00:00,389.0,POS,2017,1,58147131.81,223.0,17.0
48803,2017-01-04,901,5814,type2,810,14APR17:00:00:00,99.0,POS,2017,1,58147131.81,223.0,17.0
48804,2017-01-04,901,5921,type2,810,20APR17:00:00:00,385.0,POS,2017,1,58147131.81,223.0,17.0


In [64]:
# Для данных пользователей необходимо спрогнозировать рассчитанную величину'target_sum'.
user_sample_df[['cl_id','target_sum']].drop_duplicates()

,cl_id,target_sum
48794,901,58147131.81
74524,1339,7079905.92
74617,1339,2876211.78
99591,1813,921600.00
134218,2414,2637756.90
152544,2715,48508409.67
261930,4977,0.00
352786,7082,0.00
359055,7213,0.00
404364,8274,151400.62
